In [ ]:
/**
隐士类型抓换
implicit这个隐式转换是支撑scala的易用、容错以及灵活语法的基础

1. 记住隐式转换函数的同一个scop中不能存在参数和返回值完全相同的2个implicit函数。
2. 隐式转换函数只在意 输入类型，返回类型，和函数名，参数名无关。
3. 隐式转换是scala的语法灵活和简洁的重要组成部分。

4. implicit 关键字可通过两种方式使用：1、方法或变量定义；2、方法参数列表


*/

In [1]:
def foo(msg : String) = println(msg)


defined function foo

In [2]:
foo("2")

2


In [3]:
foo(3)

: 

In [4]:
implicit def intToString(i: Int) = i.toString

defined function intToString

In [5]:
foo(3)

3


In [8]:
def bar(msg : String) = println("aslo can use implicit function intToString...result is "+msg)
bar(33)

aslo can use implicit function intToString...result is 33


defined function bar

In [9]:
/**
    二义性的异常，说的是intToString和int2str这2个隐式函数都是可以处理bar(33)的，编译器不知道选择哪个了
    证明了隐式函数和函数名，参数名无关，只和输入参数与返回值有关。
*/

implicit def int2str(o: Int) = o.toString
bar(33)

: 

In [ ]:
/** 隐式函数的应用
我们可以随便的打开scala函数的一些内置定义，比如我们最常用的map函数中->符号，看起来很像php等语言。
但实际上->确实是一个ArrowAssoc类的方法，它位于scala源码中的Predef.scala中。下面是这个类的定义：
final class ArrowAssoc[A](val __leftOfArrow: A) extends AnyVal {  
  // `__leftOfArrow` must be a public val to allow inlining. The val  
  // used to be called `x`, but now goes by `__leftOfArrow`, as that  
  // reduces the chances of a user's writing `foo.__leftOfArrow` and  
  // being confused why they get an ambiguous implicit conversion  
  // error. (`foo.x` used to produce this error since both  
  // any2Ensuring and any2ArrowAssoc pimped an `x` onto everything)  
  @deprecated("Use `__leftOfArrow` instead", "2.10.0")  
  def x = __leftOfArrow  
  
  @inline def -> [B](y: B): Tuple2[A, B] = Tuple2(__leftOfArrow, y)  
  def →[B](y: B): Tuple2[A, B] = ->(y)  
}  
@inline implicit def any2ArrowAssoc[A](x: A): ArrowAssoc[A] = new ArrowAssoc(x)  


*/

我们看到def ->[B] (y :B)返回的其实是一个Tuple2[A,B]类型。
我们定义一个Map:
scala> val mp = Map(1->"game1",2->"game_2")  
mp: scala.collection.immutable.Map[Int,String] = Map(1 -> game1, 2 -> game_2)  
这里 1->"game1"其实是1.->("game_1")的简写。
这里怎么能让整数类型1能有->方法呢。
这里其实any2ArrowAssoc隐式函数起作用了，这里接受的参数[A]是泛型的，所以int也不例外。
调用的是：将整型的1 implicit转换为 ArrowAssoc(1)
看下构造方法，将1当作__leftOfArrow传入。
->方法的真正实现是生产一个Tuple2类型的对象(__leftOfArrow,y ) 等价于(1, "game_id")
这就是一个典型的隐式转换应用。